In [21]:
import pandas as pd
from datetime import datetime
from slugify import slugify

In [61]:
df = pd.read_csv('csv/products.csv')
priceDf = pd.read_csv('out/price.csv')
usersDf = pd.read_csv('csv/users.csv')
users = pd.read_csv('out/users.csv')
locDf = pd.read_csv('out/localities.csv', delimiter=';')
prodDf = pd.read_csv('out/products.csv')

In [62]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [63]:
df1 = df[['Price 1', 'Curr1', 'Number Review 1', 'Review 1', 'Rating 1', 'ID', 'Name']]
df2 = df[['Price 2', 'Curr2', 'Number Review 2', 'Review 2', 'Rating 2', 'ID', 'Name']]
df3 = df[['Price 3', 'Curr3', 'Number Review 3', 'Review 3', 'Rating 3', 'ID', 'Name']]
df4 = df[['Price 4', 'Curr4', 'Number Review 4', 'Review 4', 'Rating 4', 'ID', 'Name']]
df5 = df[['Price 5', 'Curr5', 'Number Review 5', 'Review 5', 'Rating 5', 'ID', 'Name']]

In [64]:
df1 = df1.rename(columns={'Price 1' : 'Price', 'Curr1' : 'Curr', 'Number Review 1' : 'Number Review', \
                          'Review 1' : 'Review', 'Rating 1' : 'Rating'})
df2 = df2.rename(columns={'Price 2' : 'Price', 'Curr2' : 'Curr', 'Number Review 2' : 'Number Review', \
                          'Review 2' : 'Review', 'Rating 2' : 'Rating'})
df3 = df3.rename(columns={'Price 3' : 'Price', 'Curr3' : 'Curr', 'Number Review 3' : 'Number Review', \
                          'Review 3' : 'Review', 'Rating 3' : 'Rating'})
df4 = df4.rename(columns={'Price 4' : 'Price', 'Curr4' : 'Curr', 'Number Review 4' : 'Number Review', \
                          'Review 4' : 'Review', 'Rating 4' : 'Rating'})
df5 = df5.rename(columns={'Price 5' : 'Price', 'Curr5' : 'Curr', 'Number Review 5' : 'Number Review', \
                          'Review 5' : 'Review', 'Rating 5' : 'Rating'})

In [65]:
newDf = pd.concat([df1, df2, df3, df4, df5])

In [66]:
newDf.rename(columns = {'Review':'review', 'Rating' : 'rating'}, inplace = True)

In [67]:
columns = ['id', 'createdDate', 'updatedDate', 'name', 'internalNote', 'isActive', 'authorId', 'statusId', \
          'productId', 'priceId', 'placeId', 'slug']

In [68]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [69]:
newDf['id'] = newDf.index + 1
newDf['createdDate'] = dt_string
newDf['updatedDate'] = dt_string
newDf['name'] = ''
newDf['internalNote'] = ''
newDf['isActive'] = True
newDf['statusId'] = 4
newDf['placeId'] = ''

In [70]:
def set_slug(value):
    return slugify(str(value))

newDf['slug'] = newDf['review'][:10].apply(set_slug)

In [71]:
priceDf['merged'] = priceDf['amount'] + priceDf['currencyId']
newDf['merged'] = newDf['Price'] + newDf['Curr']

In [72]:
def get_price_id(value):
    try:
        return priceDf[:][priceDf['merged'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['priceId'] = newDf['merged'].apply(get_price_id)

In [73]:
def get_product_id(value):
    try:
        return prodDf[:][prodDf['name'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['productId'] = newDf['Name'].apply(get_product_id)

In [95]:
def get_author_id(value):
    try:
        temp = usersDf[:][(usersDf['Review1'] == str(value)) | \
         (usersDf['Review2'] == str(value)) | \
         (usersDf['Review3'] == str(value)) | \
         (usersDf['Review4'] == str(value)) | \
         (usersDf['Review5'] == str(value))]['Username'].values[0].replace("\n", "")
        return users[:][users['username'] == temp]['id'].values[0]
    except:
        return ''

newDf['authorId'] = newDf['Number Review'].apply(get_author_id)

In [98]:
newDf.to_csv('out/stories.csv', index=False, encoding='utf-8')